[This document on GitHub](https://github.com/scaleoutsean/awesome-solidfire/tree/master/scripts/Managing-Solidfire.ipynb)

# Managing SolidFire using PowerShell in .NET Interactive Notebook

This simple demo shows how to use PowerShell in .NET Interactive Notebook using cmdlets from [SolidFire PowerShell](https://github.com/solidfire/PowerShell) package against SolidFire v12 (any release between 11 and 12.3 should work). 

This Notebook was tested twice and seems to work fine, but if you encounter any issues you can fix them on your own.

This Notebook is licensed under the MIT License, same as [.NET Interactive](https://github.com/dotnet/interactive).

## Prerequisites

- Microsoft Code 1.65 or newer
- .NET 6.0 SDK or runtime (Windows, Linux, OS X)
- Microsoft Code extension: .NET Interactive Notebook

## Apply environment variables

Go to the bottom of this notebook, edit and then apply environment variables required to connect to SolidFire cluster management IP. S3-related variables can be left alone if you won't try to use that feature.

## Workflow

Install PowerShell module SolidFire.Core:

In [ ]:
Install-Module SolidFire.Core -Force

Import module and connect to your SolidFire cluster:

In [ ]:
Import-Module SolidFire.Core 
Connect-SFCluster $MVIP -Username $CLUSTERADMIN -Password $PASSWORD


Target           : 192.168.1.34
Name             : DR
Port             : 
VersionApiName   : Magnesium
VersionApiNumber : 12.3
Node             : False
Uri              : https://192.168.1.34/json-rpc/12.3
RequestCount     : 3
Credential       : System.Management.Automation.PSCredential
Limits           : {"AccountCountMax" = 20, "AccountNameLengthMax" = 64, "AccountNameLengthMin" = 
                   1, "BulkVolumeJobsPerNodeMax" = 10, "BulkVolumeJobsPerVolumeMax" = 10, 
                   "CloneJobsPerVolumeMax" = 1, "ClusterPairsCountMax" = 4, 
                   "InitiatorNameLengthMax" = 224, "InitiatorCountMax" = 10, 
                   "InitiatorsPerVolumeAccessGroupCountMax" = 10, 
                   "IscsiSessionsFromFibreChannelNodesMax" = 4096, "QosPolicyCountMax" = 10, 
                   "SecretLengthMax" = 16, "ScheduleNameLengthMax" = 244, "SecretLengthMin" = 12, 
                   "SnapshotNameLengthMax" = 255, "SnapshotsPerVolumeMax" = 10, 
                   "Volum

## Get cluster capacity

Get cluster capacity and ... do something with it:

In [ ]:
$cluster_cap = Get-SFClusterCapacity

Compute stuff:

In [ ]:
# Adopted and style-modified (formulae are the same) by scaleoutSean from:
# https://github.com/kpapreck/test-plan/blob/master/sf-efficiency.ps1

#REQUIRED INPUTS
$nonZeroBlocks = $cluster_cap | Select-Object nonZeroBlocks -ExpandProperty nonZeroBlocks
$zeroBlocks = $cluster_cap | Select-Object zeroBlocks -ExpandProperty zeroBlocks
$uniqueBlocks = $cluster_cap | Select-Object uniqueBlocks -ExpandProperty uniqueBlocks
$uniqueBlocksUsedSpace = $cluster_cap | Select-Object uniqueBlocksUsedSpace -ExpandProperty uniqueBlocksUsedSpace
$snapshotNonZeroBlocks = $cluster_cap | Select-Object snapshotNonZeroBlocks -ExpandProperty snapshotNonZeroBlocks

#EFFICIENCY CALCULATIONS
[single]$thinProvisioningFactor = (($nonZeroBlocks+$zeroBlocks)/$nonZeroBlocks)
[single]$deDuplicationFactor = (($nonZeroBlocks+$snapshotNonZeroBlocks)/$uniqueBlocks)
[single]$compressionFactor = (($uniqueBlocks*4096)/($uniqueBlocksUsedSpace*.93))

#FOR DEBUGGING
#echo $thinProvisioningFactor
#echo $deDuplicationFactor
#echo $compressionFactor

#CALCULATE EFFICIENCY FACTOR FOR COMPRESSION + DEDUPLICATION ONLY
$efficiencyFactor = ($deDuplicationFactor*$compressionFactor)

#CALCULATE FULL EFFICIENCY FACTOR FOR COMPRESSION + DEDUPLICATION + THIN PROVISIONING
$efficiencyFullFactor = ($deDuplicationFactor*$compressionFactor*$thinProvisioningFactor)

#GET THE CLUSTER ERROR THRESHOLD BYTES
$cluster_full_threshold = Get-SfClusterfullthreshold
$errorThreshold = $cluster_full_threshold | Select-Object stage4BlockThresholdBytes -ExpandProperty Stage4BlockThresholdBytes
$errorThresholdTB = ($errorThreshold/1000/1000/1000/1000)

#GET THE TOTAL USED RAW CAPACITY
$sumUsedCapacity = $cluster_full_threshold | Select-Object sumUsedClusterBytes -ExpandProperty sumUsedClusterBytes
$sumUsed = ($sumUsedCapacity/1000/1000/1000/1000)

#DETERMINE THE RAW SPACE AVAILABLE ON THE CLUSTER UNTIL ERROR THRESHOLD
$rawSpaceAvailableTB = (($errorThreshold-$sumUsedCapacity)/(1000*1000*1000*1000))

#DETERMINE THE RAW SPACE AVAILABLE ON THE CLUSTER UNTIL 100% FULL
$stage5BlockThresholdBytes = $cluster_full_threshold | Select-Object stage5BlockThresholdBytes -ExpandProperty stage5BlockThresholdBytes
$rawSpaceAvailable100TB = (($stage5BlockThresholdBytes-$sumUsedCapacity)/(1000*1000*1000*1000))

#GET TOTAL CLUSTER CAPCITY
$sumTotalClusterBytes = $cluster_full_threshold | Select-Object sumTotalClusterBytes -ExpandProperty sumTotalClusterBytes
$sumTotalClusterBytes = ($sumTotalClusterBytes/1000/1000/1000/1000)

#GET CLUSTER FULL EFFECTIVE
$sumClusterFulldc = ($sumTotalClusterBytes*$efficiencyFactor)/2
$sumClusterFulldct = ($sumTotalClusterBytes*$efficiencyFullFactor)/2

#GET THE EFFECTIVE CAPACITY REMAINING OF COMPRESSION + DEDUPLICATION UNTIL ERROR THRESHOLD
$effectiveCapacityRemaining = ($rawSpaceAvailableTB*$efficiencyFactor)/2

#GET THE EFFECTIVE CAPACITY OF COMPRESSION + DEDUPLICATION + THIN PROVISIONING UNTIL ERROR THRESHOLD
$effectiveFullCapacityRemaining = ($rawSpaceAvailableTB*$efficiencyFullFactor)/2

#GET THE EFFECTIVE CAPACITY REMAINING OF COMPRESSION + DEDUPLICATION UNTIL 100% FULL
$effectiveCapacityRemaining100 = ((($stage5BlockThresholdBytes-$sumUsedCapacity)*$efficiencyFactor)/(1000*1000*1000*1000))/2

#GET CLUSTER NAME
$clusterName = Get-SfClusterInfo | Select-Object name -ExpandProperty name

#FORMAT TO 2 DECIMALS
$efficiencyFactor = "{0:N2}" -f $efficiencyFactor
$efficiencyFullFactor = "{0:N2}" -f $efficiencyFullFactor
$rawSpaceAvailableTB = "{0:N2}" -f $rawSpaceAvailableTB
$effectiveCapacity = "{0:N2}" -f $effectiveCapacity
$sumTotalClusterBytes = "{0:N2}" -f $sumTotalClusterBytes
$effectiveCapacityRemaining = "{0:N2}" -f $effectiveCapacityRemaining
$sumClusterFulldc = "{0:N2}" -f $sumClusterFulldc
$sumClusterFulldct = "{0:N2}" -f $sumClusterFulldct
$effectiveCapacityRemaining100 = "{0:N2}" -f $effectiveCapacityRemaining100
$sumUsed = "{0:N2}" -f $sumUsed
$errorThresholdTB = "{0:N2}" -f $errorThresholdTB
$compressionFactor = "{0:N2}" -f $compressionFactor
$deDuplicationFactor = "{0:N2}" -f $deDuplicationFactor
$thinProvisioningFactor = "{0:N2}" -f $thinProvisioningFactor
$rawSpaceAvailable100TB = "{0:N2}" -f $rawSpaceAvailable100TB
$effectiveFullCapacityRemaining = "{0:N2}" -f $effectiveFullCapacityRemaining

Write-Host "--------------------------------------------------------------------------------------------------------------"
Write-Host "SolidFire Cluster: $clusterName"
Write-Host ""
Write-Host "Cluster RAW Capacity: $sumTotalClusterBytes TB"
Write-Host "Cluster RAW Capacity Error Stage: $errorThresholdTB TB"
Write-Host "Cluster RAW Capacity Used: $sumUsed"
Write-Host "Cluster Error Stage RAW TB Remaining Available: $rawSpaceAvailableTB TB"
Write-Host "Cluster 100% Full RAW TB Remaining Available: $rawSpaceAvailable100TB TB"

Write-Host ""
Write-Host "Cluster Efficiencies"
Write-Host "Thin Provisioning Ratio: $thinProvisioningFactor"
Write-Host "Deduplication Ratio: $deDuplicationFactor"
Write-Host "Compression Ratio: $compressionFactor"
Write-Host "Cluster Deduplication/Compression Efficiency: $efficiencyFactor"
Write-host "Cluster Deduplication/Compression/Thin Provisioning Efficiency: $efficiencyFullFactor"
Write-Host ""
Write-Host "Cluster Capacity"
Write-Host "Cluster Effective Capacity @ 100% Full with Dedup/Comp: $sumClusterFulldc TB"
Write-Host "Cluster Effective Capacity @ 100% Full with Dedup/Comp/Thin: $sumClusterFulldct TB"
Write-Host "Effective Capacity Remaining until 100% Full with Dedup/Comp: $effectiveCapacityRemaining100 TB"
Write-Host "Effective Capacity Remaining until Error Threshold with Dedup/Comp: $effectiveCapacityRemaining TB"
Write-Host "Effective Capacity Remaining until Error Threshold with Dedup/Comp/Thin: $effectiveFullCapacityRemaining TB"
Write-Host "--------------------------------------------------------------------------------------------------------------"

--------------------------------------------------------------------------------------------------------------
SolidFire Cluster: DR

Cluster RAW Capacity: 0.06 TB
Cluster RAW Capacity Error Stage: 0.04 TB
Cluster RAW Capacity Used: 0.00
Cluster Error Stage RAW TB Remaining Available: 0.04 TB
Cluster 100% Full RAW TB Remaining Available: 0.06 TB

Cluster Efficiencies
Thin Provisioning Ratio: 25.03
Deduplication Ratio: 1.03
Compression Ratio: 1.67
Cluster Deduplication/Compression Efficiency: 1.71
Cluster Deduplication/Compression/Thin Provisioning Efficiency: 42.86

Cluster Capacity
Cluster Effective Capacity @ 100% Full with Dedup/Comp: 0.06 TB
Cluster Effective Capacity @ 100% Full with Dedup/Comp/Thin: 1.38 TB
Effective Capacity Remaining until 100% Full with Dedup/Comp: 0.05 TB
Effective Capacity Remaining until Error Threshold with Dedup/Comp: 0.03 TB
Effective Capacity Remaining until Error Threshold with Dedup/Comp/Thin: 0.87 TB
--------------------------------------------------

## Backup volume to S3

Use built-in backup feature to backup a volume to S3. 

Get volume information first.

In [ ]:
$vol_data     = Get-SFVolume -VolumeID $VOLID
$cluster_info = Get-SFClusterInfo

Use obtained data and environment variables to spit out some info and backup volume to S3:

In [ ]:
$PREFIX     = $cluster_info.Name + "-" + $cluster_info.UniqueID + "/" + $vol_data.Name + "-" + $vol_data.VolumeID
Write-Host "Logical volume size in 4 KiB blocks:" ($vol_data.TotalSize/$vol_data.BlockSize)
Write-Host "Initiating backup to:" $PREFIX
$backup_job = Invoke-SFApi -Method StartBulkVolumeRead `
  -Params @{ "volumeID"= "$VOLID"; "format" = "native"; `
  "script" = "bv_internal.py"; "scriptParameters" = `
  @{ "write" = @{ "awsAccessKeyID" = $S3_ACCESS_KEY; `
  "awsSecretAccessKey" = $S3_SECRET_KEY; `
  "bucket"= $S3_BUCKET; `
  "prefix"= $PREFIX; `
  "endpoint"= "s3"; "hostname"= $S3_ENDPOINT }}} 
Write-Host  = "Backup job handle:" $backup_job.asyncHandle

Logical volume size in 4 KiB blocks: 524288
Initiating backup to: DR-46z9/etcd3-4
= Backup job handle: 36


Check progress of async job handle from backup job (if job was successfully submitted and if you execute this step while backup job is still running):

In [ ]:
Get-SFASyncResult -ASyncResultID $backup_job.asyncHandle


Name                           Value
----                           -----
lastUpdateTime                 3/29/2022 8:44:08 AM
createTime                     3/29/2022 8:44:05 AM
status                         running
details                        {bvID, message, volumeID}
resultType                     BulkVolume



### Setup common variables

These variables are used throughout the notebook so set them at the top so they can be used everywhere.

> NOTE: This also means that all you have to do is change the vaules here and the Notebook will just work still (so long as the values are correct)

In [ ]:
# SolidFire credentials
$CLUSTERADMIN = 'admin'
$PASSWORD     = 'dummy-pass'

# S3 credentials
$S3_ACCESS_KEY= ''
$S3_SECRET_KEY= ''

# SolidFire resource details
$MVIP         = '192.168.1.34'
$VOLID        = 4
# S3 endpoint
$S3_ENDPOINT  = 's3.com.org'
$S3_BUCKET    = 'solidfire-native-backup'

